In [ ]:
from flask import Flask, request, jsonify
import os
import shutil
import argparse
import yt_dlp
from moviepy.editor import VideoFileClip, concatenate_audioclips
import yagmail

app = Flask(__name__)

@app.route('/generate_mashup', methods=['POST'])
def generate_mashup():
    try:
        data = request.json

        singer = data.get('singer')
        n = int(data.get('n'))
        duration = int(data.get('duration'))
        email = data.get('email')

        if n <= 10 or duration <= 20:
            return jsonify({"error": "Please provide values greater than 10 for N and 20 for duration."}), 400

        os.makedirs("downloads", exist_ok=True)

        videos = download_videos(singer, n)

        audio_files = []
        for i in range(1, n+1):
            video_path = f"downloads/{singer}-{i}.mp4"
            audio_path = f"downloads/{singer}-{i}.mp3"
            cut_audio_path = f"downloads/{singer}-{i}_cut.mp3"

            convert_to_audio(video_path, audio_path)
            cut_audio(audio_path, cut_audio_path, duration)

            audio_files.append(cut_audio_path)

        output_path = f"downloads/{singer}_output.mp3"
        merge_audios(audio_files, output_path)

        zip_file_path = f"downloads/{singer}_mashup.zip"
        shutil.make_archive(zip_file_path[:-4], 'zip', "downloads", f"{singer}_output.mp3")

        # Sending email with mashup zip file
        yag = yagmail.SMTP('your_email@gmail.com', 'your_email_password')
        yag.send(
            to=email,
            subject="Mashup File",
            contents="Here is your mashup file.",
            attachments=zip_file_path
        )

        print(f"Mashup file sent to {email}")

        clean_up()

        return jsonify({"message": "Mashup file generated and sent successfully."}), 200

    except Exception as e:
        import traceback
        traceback.print_exc()
        clean_up()
        return jsonify({"error": str(e)}), 500

def download_videos(singer, n):
    url = f"https://www.youtube.com/results?search_query={singer}"
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',
        }],
        'outtmpl': f'downloads/{singer}-%(title)s.%(ext)s',
        'noplaylist': True,
        'quiet': True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=True)
        return ydl.prepare_filename(info_dict)

def convert_to_audio(video_path, output_path):
    clip = VideoFileClip(video_path)
    audio_clip = clip.audio
    audio_clip.write_audiofile(output_path)

def cut_audio(input_path, output_path, duration):
    audio_clip = AudioFileClip(input_path).subclip(0, duration)
    audio_clip.write_audiofile(output_path)

def merge_audios(audio_files, output_path):
    clips = [AudioFileClip(file) for file in audio_files]
    final_clip = concatenate_audioclips(clips)
    final_clip.write_audiofile(output_path)

def clean_up():
    shutil.rmtree("downloads")

if __name__ == "__main__":
    app.run(debug=True)
